In [14]:
# import necessary libriaries
import csv
import pandas as pd
import sqlite3
from datetime import datetime

# import dataset from csv
data = pd.read_csv('users.csv', delimiter=',')
print(data)
type(data)

# establish connection and cursor object
connection = sqlite3.connect("users.db")
cursor = connection.cursor()

       id                                 email campaign    test  created_at  \
0       1                   gchesshire0@ucoz.ru      NaN     NaN   1/26/2017   
1       2                       rbritner1@is.gd      NaN     NaN   10/8/2017   
2       3                 acoysh2@canalblog.com      NaN     NaN    9/5/2017   
3       4                    rbasley3@google.cn      NaN     NaN   7/29/2017   
4       5                     kdury4@elpais.com      NaN     NaN   5/28/2017   
..    ...                                   ...      ...     ...         ...   
995   996           njeffcoatrn@photobucket.com      NaN     NaN   2/21/2017   
996   997  tfeldbergerro@networkadvertising.org      NaN     NaN   3/28/2017   
997   998            driggeyrp@chronoengine.com      NaN     NaN   9/21/2017   
998   999                jgeraudyrq@cornell.edu      NaN     NaN   7/12/2017   
999  1000                nfersonrr@berkeley.edu      NaN  tigers  10/19/2017   

      birthday  
0    4/17/1982  
1    

In [15]:
# create empty table
cursor.execute ('''CREATE TABLE users(id INTEGER,
email TEXT,
campaign TEXT,
test TEXT,
created_at TEXT,
birthday TEXT);''');

In [16]:
# import dataset to database 'users' 
data.to_sql('users', connection, if_exists='replace')

In [17]:
# check if database is valid
cursor.execute("SELECT * FROM users LIMIT 10").fetchall()

[(0, 1, 'gchesshire0@ucoz.ru', None, None, '1/26/2017', '4/17/1982'),
 (1, 2, 'rbritner1@is.gd', None, None, '10/8/2017', '1/16/1970'),
 (2, 3, 'acoysh2@canalblog.com', None, None, '9/5/2017', '8/11/1986'),
 (3, 4, 'rbasley3@google.cn', None, None, '7/29/2017', '8/29/1960'),
 (4, 5, 'kdury4@elpais.com', None, None, '5/28/2017', '10/2/1988'),
 (5, 6, 'ssnowball5@ucoz.com', None, None, '11/23/2017', '6/13/1962'),
 (6, 7, 'dbagshaw6@elpais.com', None, None, '11/17/2017', '12/17/1963'),
 (7, 8, 'tdurden7@usgs.gov', None, 'bears', '4/9/2017', '8/12/1989'),
 (8, 9, 'rtinn8@archive.org', None, None, '1/14/2017', '11/8/1960'),
 (9, 10, 'styndall9@vimeo.com', None, 'bears', '11/5/2017', '1/20/1976')]

In [18]:
# save the current progress
connection.commit()

In [19]:
# create two different lists: one for date of registration the other for birth date.
result_creation_date = cursor.execute("SELECT created_at FROM users;").fetchall()
result_birthdate = cursor.execute("SELECT birthday FROM users;").fetchall()

In [20]:
# create an empty list, iterate through given lists to calculate an age of a user and append the empty list with ages
age_list = []
for a, b in zip(result_creation_date, result_birthdate):
    today = datetime.strptime(*a, '%m/%d/%Y').date() # take variable from tuple and convert it from string to date format
    birthday = datetime.strptime(*b, '%m/%d/%Y').date() # the same operation for birhdays
    age = today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day)) # age calculation
    age_list.append(age) # append each calculation result to a list
print (age_list) 

[34, 47, 31, 56, 28, 55, 53, 27, 56, 41, 32, 53, 49, 54, 56, 50, 33, 32, 33, 28, 51, 36, 28, 47, 29, 57, 35, 28, 54, 38, 49, 46, 42, 42, 28, 38, 40, 41, 40, 29, 41, 30, 28, 46, 28, 30, 40, 44, 51, 43, 33, 44, 54, 47, 55, 49, 29, 45, 55, 31, 51, 54, 48, 54, 29, 34, 29, 42, 28, 48, 43, 33, 48, 45, 31, 28, 36, 29, 35, 29, 37, 34, 48, 43, 54, 49, 36, 35, 39, 44, 35, 50, 33, 53, 30, 30, 44, 37, 35, 54, 34, 36, 52, 30, 28, 27, 43, 37, 53, 48, 36, 51, 29, 43, 49, 46, 35, 43, 44, 56, 43, 31, 28, 55, 53, 28, 49, 47, 52, 32, 43, 30, 49, 52, 48, 50, 56, 52, 36, 54, 55, 37, 35, 33, 39, 28, 36, 53, 51, 29, 32, 37, 51, 31, 33, 30, 52, 48, 30, 50, 32, 39, 48, 33, 43, 38, 52, 49, 30, 27, 37, 32, 29, 40, 57, 44, 49, 53, 38, 39, 51, 35, 56, 45, 29, 30, 34, 30, 33, 41, 56, 54, 33, 37, 40, 42, 44, 43, 39, 30, 53, 48, 27, 27, 31, 55, 40, 43, 30, 54, 27, 35, 51, 40, 57, 40, 56, 41, 31, 37, 52, 36, 45, 31, 49, 34, 45, 40, 47, 53, 55, 37, 54, 50, 54, 33, 50, 30, 55, 42, 53, 57, 56, 56, 35, 35, 35, 55, 53, 46,

In [21]:
# add to the table a new column 'age'
cursor.execute ('''ALTER TABLE users
ADD COLUMN age INTEGER;''')

In [24]:
# add data to the age column from age_list
cursor.executemany('UPDATE users SET age= ?', ((i,) for i in age_list))

In [25]:
# check that we get a required result
cursor.execute("SELECT * FROM users LIMIT 10").fetchall()

[(0, 1, 'gchesshire0@ucoz.ru', None, None, '1/26/2017', '4/17/1982', 54),
 (1, 2, 'rbritner1@is.gd', None, None, '10/8/2017', '1/16/1970', 54),
 (2, 3, 'acoysh2@canalblog.com', None, None, '9/5/2017', '8/11/1986', 54),
 (3, 4, 'rbasley3@google.cn', None, None, '7/29/2017', '8/29/1960', 54),
 (4, 5, 'kdury4@elpais.com', None, None, '5/28/2017', '10/2/1988', 54),
 (5, 6, 'ssnowball5@ucoz.com', None, None, '11/23/2017', '6/13/1962', 54),
 (6, 7, 'dbagshaw6@elpais.com', None, None, '11/17/2017', '12/17/1963', 54),
 (7, 8, 'tdurden7@usgs.gov', None, 'bears', '4/9/2017', '8/12/1989', 54),
 (8, 9, 'rtinn8@archive.org', None, None, '1/14/2017', '11/8/1960', 54),
 (9, 10, 'styndall9@vimeo.com', None, 'bears', '11/5/2017', '1/20/1976', 54)]

In [26]:
cursor.close()
connection.commit()
connection.close()